In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import numpy as np
from pyuvdata import UVCal, UVData, utils
import os
import sys
import glob
import uvtools as uvt
from astropy.time import Time
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, Angle
import pandas
import warnings 
import copy
import utils
from hera_mc import cm_hookup
warnings.filterwarnings('ignore')

In [ ]:
# get data location
data_path = os.environ['DATA_PATH']


In [ ]:
# Load in data
HHfiles, uvdx, uvdy, uvd1, uvd2, uvd_diff, uvd_sum = utils.load_data(data_path)

## Autocorrelations for a single file

This plot shows autocorrelations for one timestamp of each antenna that is active and each polarization

In [ ]:
### plot autos
utils.plot_autos(uvdx, uvdy, uvd1, uvd2)

## Waterfalls of Autocorrelation Amplitudes for each Antenna and Each polarization

In [ ]:
uvd = UVData()
uvd.read_uvh5(HHfiles, ant_str = 'autos')
lst0 = np.min(uvd1.lst_array) 
lst1 = np.max(uvd2.lst_array)

In [ ]:
utils.plot_wfs(uvd, pol = 0)

In [ ]:
utils.plot_wfs(uvd, pol = 1)

## Correlation Matrices

In [ ]:
matrix = utils.calcEvenOddAmpMatrix(uvd_sum,uvd_diff,nodes='auto')
utils.plotCorrMatrix(uvd_sum, matrix, nodes='auto')